In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import torch
print(torch.__version__)
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset
from torchvision import transforms as T
import random
import cv2
import numpy as np
!pip install pyefd
import pyefd
from google.colab.patches import cv2_imshow
!pip install cairocffi
import cairocffi as cairo
from struct import pack, unpack, error

1.12.1+cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 6.1 MB/s 
  Created wheel for cairocffi: filename=cairocffi-1.3.0-py3-none-any.whl size=89668 sha256=ab0f38ad88f84afe5abea9acbfc46b00f2407c7023f5b15143e2440d9399ad60
  Stored in directory: /root/.cache/pip/wheels/4e/ca/e1/5c8a9692a27f639a07c949044bec943f26c81cd53d3805319f
Successfully built cairocffi


In [7]:
# const vars
train_dir = '/content/drive/My Drive/Fourier/Quickdraw Dataset Small/Train/'
test_dir = '/content/drive/My Drive/Fourier/Quickdraw Dataset Small/Test/'
FOURIER_ORDER = 20
IMG_SIDE = 256
RAND_SEED = 0
num_train = 1000
num_test = 100
list_of_classes = ["arm", "beard", "brain", "circle", "ear", "elbow", "eye", 
                   "face", "finger", "foot", "goatee", "hand", "hexagon", "knee", 
                   "leg", "line", "moustache", "mouth", "nose", "octagon", 
                   'skull', 'square', 'squiggle', 'star', 'toe', 'tooth', 
                   'triangle', 'zigzag']
# list_of_classes = ["The Eiffel Tower", "The Great Wall of China", "The Mona Lisa",
#                    "aircraft carrier", "airplane", "alarm clock", "ambulance", 
#                    "angel", "ant", "anvil", "apple", "arm", "asparagus", "axe", 
#                    "backpack", "banana", "bandage", "barn", "baseball bat", 
#                    "baseball", "basket", "basketball", "bathtub", "beach", "bear", 
#                    "beard", "bed", "bee", "belt", "bench", "bicycle", "binoculars", 
#                    "bird", "birthday cake", "blackberry", "blueberry", "book", 
#                    "boomerang", "bottlecap", "bowtie", "bracelet", "brain", 
#                    "bread", "bridge", "broccoli", "broom", "bucket", "bulldozer", 
#                    "bus", "bush", "butterfly", "cactus", "cake", "calculator", 
#                    "calendar", "camel", "camera", "camouflage", "campfire", 
#                    "candle", "cannon", "canoe", 'car', 'carrot', "castle", "cat", "ceiling fan", 
#                    "cell phone", "cello", "chair", "chandelier", "church", 
#                    "circle", "clarinet", "clock", "cloud", "coffee cup", 
#                    "compass", "computer", "cookie", "cooler", "couch", "cow",
#                    "crab", "crayon", "crocodile", "crown", "cruise ship", 
#                    "cup", "diamond", "dishwasher", "diving board", "dog", 
#                    "dolphin", "donut", "door", "dragon", "dresser", "drill", 
#                    "drums", "duck", "dumbbell", "ear", "elbow", "elephant", 
#                    "envelope", "eraser", "eye", "eyeglasses", "face", "fan",
#                    "feather", "fence", "finger", "fire hydrant", "fireplace",
#                    "firetruck", "fish", "flamingo", "flashlight", "flip flops", 
#                    "floor lamp", "flower", "flying saucer", "foot", "fork", 
#                    "frog", "frying pan", "garden hose", "garden", "giraffe", 
#                    "goatee", "golf club", "grapes", "grass", "guitar", 
#                    "hamburger", "hammer", "hand", "harp", "hat", "headphones", 
#                    "hedgehog", "helicopter", "helmet", "hexagon", "hockey puck", 
#                    "hockey stick", "horse", "hospital", "hot air balloon", 
#                    "hot dog", "hot tub", "hourglass", "house plant", "house", 
#                    "hurricane", "ice cream", "jacket", "jail", "kangaroo", 
#                    "key", "keyboard", "knee", "knife", "ladder", "lantern", 
#                    "laptop", "leaf", "leg", "light bulb", "lighter", "lighthouse",
#                    "lightning", "line", "lion", "lipstick", "lobster", "lollipop",
#                    "mailbox", "map", "marker", "matches", "megaphone", "mermaid", 
#                    "microphone", "microwave", "monkey", "moon", "mosquito", 
#                    "motorbike", "mountain", "mouse", "moustache", "mouth", "mug",
#                    "mushroom", "nail", "necklace", "nose", "ocean", "octagon", 
#                    "octopus", "onion", "oven", "owl", "paint can", "paintbrush", 
#                    "palm tree", "panda", "pants", "paper clip", "parachute", 
#                    "parrot", "passport", "peanut", "pear", "peas", "pencil", 
#                    "penguin", "piano", "pickup truck", "picture frame", "pig", 
#                    "pillow", "pineapple", "pizza", "pliers", "police car", 
#                    "pond", "pool", "popsicle", "postcard", "potato", 
#                    "power outlet", "purse", "rabbit", "raccoon", "radio", 
#                    "rain", 'rainbow', 'rake', 'remote control', 'rhinoceros', 
#                    'rifle', 'river', 'roller coaster', 'rollerskates', 
#                    'sailboat', 'sandwich', 'saw', 'saxophone', 'school bus', 
#                    'scissors', 'scorpion', 'screwdriver', 'sea turtle', 
#                    'see saw', 'shark', 'sheep', 'shoe', 'shorts', 'shovel', 
#                    'sink', 'skateboard', 'skull', 'skyscraper', 'sleeping bag', 
#                    'smiley face', 'snail', 'snake', 'snorkel', 'snowflake', 
#                    'snowman', 'soccer ball', 'sock', 'speedboat', 'spider', 
#                    'spoon', 'spreadsheet', 'square', 'squiggle', 'squirrel', 
#                    'stairs', 'star', 'steak', 'stereo', 'stethoscope', 'stitches', 
#                    'stop sign', 'stove', 'strawberry', 'streetlight', 
#                    'string bean', 'submarine', 'suitcase', 'sun', 'swan', 
#                    'sweater', 'swing set', 'sword', 'syringe', 't-shirt', 
#                    'table', 'teapot', 'teddy-bear', 'telephone', 'television', 
#                    'tennis racquet', 'tent', 'tiger', 'toaster', 'toe', 'toilet', 
#                    'tooth', 'toothbrush', 'toothpaste', 'tornado', 'tractor', 
#                    'traffic light', 'train', 'tree', 'triangle', 'trombone', 
#                    'truck', 'trumpet', 'umbrella', 'underwear', 'van', 'vase', 'violin', 
#                    'washing machine', 'watermelon', 'waterslide', 'whale', 
#                    'wheel', 'windmill', 'wine bottle', 'wine glass', 'wristwatch', 
#                    'yoga', 'zebra', 'zigzag']

# define methods for unpacking Quickdraw .bin files
def unpack_drawing(file_handle):
  file_handle.read(15)
  n_strokes, = unpack('H', file_handle.read(2))
  image = []
  for i in range(n_strokes):
      n_points, = unpack('H', file_handle.read(2))
      fmt = str(n_points) + 'B'
      x = unpack(fmt, file_handle.read(n_points))
      y = unpack(fmt, file_handle.read(n_points))
      image.append((x, y))

  return image

def unpack_drawings(filename):
  drawings = []
  with open(filename, 'rb') as f:
      while True:
          try:
              drawings.append(unpack_drawing(f))
          except error:
              break
  return drawings

def sample_drawings(drawings, num_train, num_test):
  all_sampled = random.sample(drawings, k=(num_train+num_test))
  train = all_sampled[:num_train]
  test = all_sampled[num_train:]
  return train, test

def pack_drawings(filename, drawings):
  with open(filename, 'wb') as f:
    for drawing in drawings:
      f.write(pack('15x'))
      num_strokes = len(drawing)
      f.write(pack('H', num_strokes))
      for stroke in drawing:
        stroke_len = len(stroke[0])
        f.write(pack('H', stroke_len))
        fmt = str(stroke_len) + 'B'
        f.write(pack(fmt, *stroke[0]))
        f.write(pack(fmt, *stroke[1]))

In [ ]:
# download, sample, and create dataset
for item in list2:
  url = 'gs://quickdraw_dataset/full/binary/' + item + '.bin'
  dest = '/' + item + '.bin'
  !gsutil -m cp "$url" "$dest"
  drawings = unpack_drawings(dest)
  train, test = sample_drawings(drawings, num_train, num_test)

  train_file = train_dir + item + '.bin'
  test_file = test_dir + item + '.bin'
  pack_drawings(train_file, train)
  pack_drawings(test_file, test)
  !rm "$dest"

In [ ]:
def get_step(deltaX, deltaY):
  angle = np.arctan2(deltaY, deltaX)
  pi_over_8 = np.pi / 8
  if -pi_over_8 <= angle < pi_over_8:
    return (1, 0)
  elif pi_over_8 <= angle < 3 * pi_over_8:
    return (1, 1)
  elif 3 * pi_over_8 <= angle < 5 * pi_over_8:
    return (0, 1)
  elif 5 * pi_over_8 <= angle < 7 * pi_over_8:
    return (-1, 1)
  elif angle >= 7 * pi_over_8 or angle < -7 * pi_over_8:
    return (-1, 0)
  elif -7 * pi_over_8 <= angle < -5 * pi_over_8:
    return (-1, -1)
  elif -5 * pi_over_8 <= angle < -3 * pi_over_8:
    return (0, -1)
  else:
    return (1, -1)

def complete_contour(vector_img):
  complete_contour = []
  partial_contour = [point for stroke in vector_img for point in zip(*stroke)]
  if len(partial_contour) > 0:
    partial_contour.append(partial_contour[0])
    for i in range(len(partial_contour) - 1):
      x0, y0 = partial_contour[i]
      x1, y1 = partial_contour[i+1]
      if (x0, y0) == (x1, y1):
        continue
      complete_contour.append([x0, y0])
  return np.asarray(complete_contour)

# transform functions - take sketch image, return torch tensor of descriptors
def fourier_transform(vector_img, is_test):
  contour = complete_contour(vector_img)

  # to-do
  # if is_test:
  
  coeffs = pyefd.elliptic_fourier_descriptors(contour, order=FOURIER_ORDER, normalize=True)
  return coeffs

In [8]:
# convert raw vector image to single raster image
def vector_to_raster(vector_image, side=IMG_SIDE, line_diameter=16, padding=80, bg_color=(0,0,0), fg_color=(1,1,1)):
  """
  padding and line_diameter are relative to the original 256x256 image.
  """
  
  original_side = 256.
  
  surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, side, side)
  ctx = cairo.Context(surface)
  ctx.set_antialias(cairo.ANTIALIAS_BEST)
  ctx.set_line_cap(cairo.LINE_CAP_ROUND)
  ctx.set_line_join(cairo.LINE_JOIN_ROUND)
  ctx.set_line_width(line_diameter)

  # scale to match the new size
  # add padding at the edges for the line_diameter
  # and add additional padding to account for antialiasing
  total_padding = padding * 2. + line_diameter
  new_scale = float(side) / float(original_side + total_padding)
  ctx.scale(new_scale, new_scale)
  ctx.translate(total_padding / 2., total_padding / 2.)
      
  bbox = np.hstack(vector_image).max(axis=1)
  offset = ((original_side, original_side) - bbox) / 2.
  offset = offset.reshape(-1,1)
  centered = [stroke + offset for stroke in vector_image]

  # clear background
  ctx.set_source_rgb(*bg_color)
  ctx.paint()

  # draw strokes, this is the most cpu-intensive part
  ctx.set_source_rgb(*fg_color)     
  for xv, yv in centered:   
    ctx.move_to(xv[0], yv[0])
    for x, y in zip(xv, yv):
        ctx.line_to(x, y)
    ctx.stroke()

  data = surface.get_data()
  raster = np.copy(np.asarray(data)[::4]).reshape(side, side)
  return raster

# Define transformation(s) to be applied to dataset-
transforms_norm = T.Compose(
      [
          T.ToTensor(), # scales integer inputs in the range [0, 255] into the range [0.0, 1.0]
      ]
  )

# transform functions - take sketch image, return torch tensor of descriptors
def transform(vector_img, is_test):
  raster = vector_to_raster(vector_img)

  # add rotations and translations at test time
  if is_test: 
    raster = transforms_norm(raster.astype(np.float32))

    angle = random.random()*60 - 30
    deltaX = random.randint(-3, 3)
    deltaY = random.randint(-3, 3)

    raster = T.functional.affine(raster, angle, [deltaX, deltaY], 1, 0,
                                 interpolation=T.InterpolationMode.BILINEAR)
    raster = np.squeeze(raster.numpy()).astype(np.uint8)
  
  raster_binary = cv2.threshold(raster, 100, 255, cv2.THRESH_BINARY)[1]
  contours, hierarchy = cv2.findContours(raster_binary, 
                                         cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  
  largest_size = 0
  largest_index = 0
  for k, contour in enumerate(contours):
      if len(contour) > largest_size:
        largest_size = len(contour)
        largest_index = k

  contour = np.asarray(contours[largest_index]).squeeze()

  coeffs = pyefd.elliptic_fourier_descriptors(contour, order=FOURIER_ORDER, normalize=True)
  return coeffs

In [9]:
# load dataset
train_imgs = []
for item in list_of_classes:
  train_folder = train_dir + item + '.bin'
  train_drawings = unpack_drawings(train_folder)
  train_imgs += train_drawings

In [10]:
coeffs = []
for img in train_imgs:
  transformed = transform(img, False)
  coeffs.append(transformed)

coeffs = np.stack(coeffs)
mean = np.mean(coeffs, axis=0)
stdev = np.std(coeffs, axis=0)
print(mean)
print('-----------------------------------')
print(stdev)

[[ 1.00000000e+00  5.14861435e-19  8.59462640e-19 -4.34051261e-01]
 [ 1.49718473e-02  1.34727903e-03 -1.84497062e-02 -1.89958526e-02]
 [-4.84771330e-02 -2.51869128e-03 -3.40720294e-03 -1.78159634e-03]
 [ 5.06983955e-03  1.59704093e-04 -2.62269206e-03  2.92415198e-03]
 [ 3.21816824e-03 -8.86234804e-04 -6.85097001e-04 -6.91081993e-03]
 [ 6.51948277e-04 -2.99653792e-04 -1.58453633e-03 -1.29729484e-03]
 [ 1.09003765e-03 -3.26678284e-04 -5.82768021e-04 -1.91625458e-03]
 [ 1.02343531e-04 -6.61961033e-06 -2.49310198e-04  9.74631489e-05]
 [ 1.44199272e-03 -2.94483190e-04 -2.90308786e-04 -1.36471836e-03]
 [ 3.69973782e-05 -2.13514517e-04 -4.55580663e-04 -2.90560724e-04]
 [ 8.84660788e-04 -5.12428761e-05 -5.93233759e-05 -2.28192971e-03]
 [-5.69101248e-05 -1.02366038e-04 -5.02431603e-05 -8.81870114e-07]
 [ 1.17968599e-03 -4.96309536e-05 -1.87174858e-04 -1.10195818e-03]
 [ 1.23174499e-04 -1.05128032e-04 -1.33797737e-04 -1.82618305e-04]
 [ 8.11483486e-04  6.19275930e-06 -5.14750577e-05 -1.10881976e

In [ ]:
# helper method to find class based on imgset index
def find_class(idx, num_list):
  class_id = 0
  sum = num_list[class_id]
  while idx >= sum:
    class_id += 1
    sum += num_list[class_id]
  return class_id

# helper func for removing bad imgs
def remove_bad_imgs(imgset, is_test):
  bad_imgs = []

  for i, img in enumerate(imgset):
    try:
      fourier_transform(img, is_test)
    except Exception as e:
      print(i)
      bad_imgs.append(i)
      print(repr(e))

  for idx in reversed(bad_imgs):
    del imgset[idx]
  
  return imgset

# remove bad imgs from dataset
random.seed(RAND_SEED)
for item in list2:
  print(item)
  folder = train_dir + item + '.bin'
  drawings = unpack_drawings(folder)
  drawings = remove_bad_imgs(drawings, False)
  pack_drawings(folder, drawings)
  folder = test_dir + item + '.bin'
  drawings = unpack_drawings(folder)
  drawings = remove_bad_imgs(drawings, True)
  pack_drawings(folder, drawings)

car
carrot
underwear
